## Reed Solomon
#### Error Correction Codes / Erasure Coding
---

In [2]:
:dep reed-solomon

In [3]:
fn encode_batch(dat: Vec<u8>) -> Vec<u8> {
    use reed_solomon::Encoder;
    let enc = Encoder::new(8);
    Vec::from(&enc.encode(&dat[..])[..])
}

In [4]:
fn decode_batch(dat_enc: Vec<u8>) -> Vec<u8> {
    use reed_solomon::Decoder;
    let dec = Decoder::new(8);
    dec.correct(&dat_enc[..], None).unwrap().data().to_owned()
}

In [5]:
let dat = b"This is a story all about how cosmic rays turned these bits upside down.".to_vec();
dat

[84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 115, 116, 111, 114, 121, 32, 97, 108, 108, 32, 97, 98, 111, 117, 116, 32, 104, 111, 119, 32, 99, 111, 115, 109, 105, 99, 32, 114, 97, 121, 115, 32, 116, 117, 114, 110, 101, 100, 32, 116, 104, 101, 115, 101, 32, 98, 105, 116, 115, 32, 117, 112, 115, 105, 100, 101, 32, 100, 111, 119, 110, 46]

In [6]:
let mut dat_enc = encode_batch(dat);
dat_enc

[84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 115, 116, 111, 114, 121, 32, 97, 108, 108, 32, 97, 98, 111, 117, 116, 32, 104, 111, 119, 32, 99, 111, 115, 109, 105, 99, 32, 114, 97, 121, 115, 32, 116, 117, 114, 110, 101, 100, 32, 116, 104, 101, 115, 101, 32, 98, 105, 116, 115, 32, 117, 112, 115, 105, 100, 101, 32, 100, 111, 119, 110, 46, 178, 226, 206, 229, 182, 14, 174, 83]

In [7]:
dat_enc[0] = 0;
dat_enc[1] = 77;
let dat_enc = dat_enc;
dat_enc

[0, 77, 105, 115, 32, 105, 115, 32, 97, 32, 115, 116, 111, 114, 121, 32, 97, 108, 108, 32, 97, 98, 111, 117, 116, 32, 104, 111, 119, 32, 99, 111, 115, 109, 105, 99, 32, 114, 97, 121, 115, 32, 116, 117, 114, 110, 101, 100, 32, 116, 104, 101, 115, 101, 32, 98, 105, 116, 115, 32, 117, 112, 115, 105, 100, 101, 32, 100, 111, 119, 110, 46, 178, 226, 206, 229, 182, 14, 174, 83]

In [8]:
let dat = decode_batch(dat_enc);
dat

[84, 104, 105, 115, 32, 105, 115, 32, 97, 32, 115, 116, 111, 114, 121, 32, 97, 108, 108, 32, 97, 98, 111, 117, 116, 32, 104, 111, 119, 32, 99, 111, 115, 109, 105, 99, 32, 114, 97, 121, 115, 32, 116, 117, 114, 110, 101, 100, 32, 116, 104, 101, 115, 101, 32, 98, 105, 116, 115, 32, 117, 112, 115, 105, 100, 101, 32, 100, 111, 119, 110, 46]

---
### Batching:
---

In [9]:
fn encode(dat: Vec<u8>) -> Vec<u8> {
    let dat_len = dat.len();
    let batch_size = 247usize;
    let batches = ((dat_len as f64)/(batch_size as f64)).ceil() as usize;
    let mut dat_enc = Vec::<u8>::with_capacity(dat_len + 8*batches);
    let mut a = 0usize;
    for _ in 0..batches {
        let b = std::cmp::min(a + batch_size, dat_len);
        let dat_slice = &dat[a..b];
        let enc_dat_slice = encode_batch(dat_slice.to_vec());
        dat_enc.extend_from_slice(&enc_dat_slice[..]);
        a += batch_size;
        a = std::cmp::min(a, dat_len);
    }
    dat_enc
}

In [10]:
fn decode(dat_enc: Vec<u8>) -> Vec<u8> {
    let dat_enc_len = dat_enc.len();
    let batch_size = 255usize;
    let batches = ((dat_enc_len as f64)/(batch_size as f64)).ceil() as usize;
    let mut dat = Vec::<u8>::with_capacity(dat_enc_len - 8*batches);
    let mut a = 0usize;
    for _ in 0..batches {
        let b = std::cmp::min(a + batch_size, dat_enc_len);
        let dat_enc_slice = &dat_enc[a..b];
        let dat_slice = decode_batch(dat_enc_slice.to_vec());
        dat.extend_from_slice(&dat_slice[..]);
        a += batch_size;
        a = std::cmp::min(a, dat_enc_len);
    }
    dat
}

In [11]:
let mut dat = std::iter::repeat(0).take(1024).collect::<Vec<u8>>();
dat[0] = 88;
dat[1] = 88;
dat[2] = 88;
dat[1000] = 77;
dat[1001] = 44;
dat[1002] = 33;
let dat = dat;
dat

[88, 88, 88, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [12]:
let mut dat_enc = encode(dat);
dat_enc

[88, 88, 88, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 195, 219, 14, 253, 94, 65, 19, 191, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [13]:
dat_enc.len()

1064

In [14]:
let dat = decode(dat_enc);
dat

[88, 88, 88, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [15]:
dat.len()

1024